In [1]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

## Gender Dataframes

In [2]:
census_file = "Data/2010Census_age_gender.csv"
census_df = pd.read_csv(census_file, skiprows=7)
census_df.drop(0, inplace=True)
census_df.drop(1, inplace=True)
census_df['Geography'] = census_df['Geography'].map(lambda x: x.lstrip('ZCTA5').rstrip('(California part only)'))
census_df

,Geography,Total population,Male,Female,Median age (years),Male Median age (years),Female Median age (years),Average Household size,Average Family size,Persons Under 5 years,Persons Under 18 years,Persons Age 21+,Persons Age 55+,Persons Age 60+,Persons Age 65+,Percent Population Female,Percent Population Less than 18,Unnamed: 17
2,89010,31,18,13,26.5,24.5,26.5,4.43,4.43,4,13,17,2,1,1,41.9%,41.9%,NaN
3,89019,69,39,30,38.3,38.5,38.0,2.88,3.93,8,18,49,17,11,6,43.5%,26.1%,NaN
4,89060,30,15,15,44.0,38.5,48.5,3.75,3.75,5,7,23,12,7,4,50.0%,23.3%,NaN
5,89061,51,23,28,47.5,40.5,50.0,2.43,2.92,2,13,38,19,16,13,54.9%,25.5%,NaN
6,89439,80,41,39,57.5,57.5,57.5,1.95,2.40,2,6,72,46,35,20,48.8%,7.5%,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1766,96148,788,425,363,38.5,37.4,39.7,2.48,3.10,53,175,584,204,136,83,46.1%,22.2%,NaN
1767,96150,"29,792","15,802","13,990",38.0,37.1,39.1,2.36,2.96,"1,714","5,965","22,572","7,305","4,799","2,975",47.0%,20.0%,NaN
1768,96155,8,5,3,46.0,42.5,60.5,2.00,3.00,1,2,6,3,3,2,37.5%,25.0%,NaN
1769,96161,"18,451","9,669","8,782",38.4,38.1,38.7,2.52,2.95,"1,175","4,200","13,765","4,081","2,652","1,494",47.6%,22.8%,NaN


In [3]:

census_df.drop(columns=['Median age (years)', 'Male Median age (years)', 'Female Median age (years)', 'Average Household size', 'Average Family size', 'Persons Under 5 years', 'Persons Under 18 years', 'Persons Age 21+', 'Persons Age 55+', 'Persons Age 60+', 'Persons Age 65+', 'Percent Population Less than 18', 'Unnamed: 17'])


,Geography,Total population,Male,Female,Percent Population Female
2,89010,31,18,13,41.9%
3,89019,69,39,30,43.5%
4,89060,30,15,15,50.0%
5,89061,51,23,28,54.9%
6,89439,80,41,39,48.8%
...,...,...,...,...,...
1766,96148,788,425,363,46.1%
1767,96150,"29,792","15,802","13,990",47.0%
1768,96155,8,5,3,37.5%
1769,96161,"18,451","9,669","8,782",47.6%


In [4]:
census_df['Percent Population Female'] = census_df['Percent Population Female'].str.rstrip('%').astype('float')


In [5]:
census_df['Female'] = pd.to_numeric(census_df['Female'], errors='coerce')
census_df['Male'] = pd.to_numeric(census_df['Male'], errors='coerce')
census_df['Total population'] = pd.to_numeric(census_df['Total population'], errors='coerce')

In [6]:
print(census_df.dtypes)

Geography                           object
Total population                   float64
Male                               float64
Female                             float64
Median age (years)                 float64
Male Median age (years)            float64
Female Median age (years)          float64
Average Household size             float64
Average Family size                float64
Persons Under 5 years               object
Persons Under 18 years              object
Persons Age 21+                     object
Persons Age 55+                     object
Persons Age 60+                     object
Persons Age 65+                     object
Percent Population Female          float64
Percent Population Less than 18     object
Unnamed: 17                        float64
dtype: object


In [7]:
census_df['Percent Population Male'] = 100 - census_df['Percent Population Female']
census_df.head(10)

,Geography,Total population,Male,Female,Median age (years),Male Median age (years),Female Median age (years),Average Household size,Average Family size,Persons Under 5 years,Persons Under 18 years,Persons Age 21+,Persons Age 55+,Persons Age 60+,Persons Age 65+,Percent Population Female,Percent Population Less than 18,Unnamed: 17,Percent Population Male
2,89010,31.0,18.0,13.0,26.5,24.5,26.5,4.43,4.43,4,13,17,2,1,1,41.9,41.9%,NaN,58.1
3,89019,69.0,39.0,30.0,38.3,38.5,38.0,2.88,3.93,8,18,49,17,11,6,43.5,26.1%,NaN,56.5
4,89060,30.0,15.0,15.0,44.0,38.5,48.5,3.75,3.75,5,7,23,12,7,4,50.0,23.3%,NaN,50.0
5,89061,51.0,23.0,28.0,47.5,40.5,50.0,2.43,2.92,2,13,38,19,16,13,54.9,25.5%,NaN,45.1
6,89439,80.0,41.0,39.0,57.5,57.5,57.5,1.95,2.40,2,6,72,46,35,20,48.8,7.5%,NaN,51.2
7,90001,NaN,NaN,NaN,26.6,26.1,27.1,4.40,4.53,"5,733","19,979","33,962","6,970","4,797","3,179",50.2,35.0%,NaN,49.8
8,90002,NaN,NaN,NaN,25.5,24.6,26.5,4.36,4.59,"5,318","18,504","29,665","6,190","4,253","2,922",51.4,36.1%,NaN,48.6
9,90003,NaN,NaN,NaN,26.3,25.5,27.1,4.22,4.45,"6,837","23,475","39,115","7,698","5,126","3,418",50.8,35.4%,NaN,49.2
10,90004,NaN,NaN,NaN,34.8,33.7,36.0,2.73,3.37,"3,972","13,295","46,400","11,720","8,158","5,419",49.7,21.4%,NaN,50.3
11,90005,NaN,NaN,NaN,33.9,32.6,35.6,2.50,3.22,"2,339","7,664","28,465","6,961","5,113","3,657",48.8,20.3%,NaN,51.2


In [8]:
df = census_df.drop(columns=['Median age (years)', 'Male Median age (years)', 'Female Median age (years)', 'Average Household size', 'Average Family size', 'Persons Under 5 years', 'Persons Under 18 years', 'Persons Age 21+', 'Persons Age 55+', 'Persons Age 60+', 'Persons Age 65+', 'Percent Population Less than 18', 'Unnamed: 17'])
df

,Geography,Total population,Male,Female,Percent Population Female,Percent Population Male
2,89010,31.0,18.0,13.0,41.9,58.1
3,89019,69.0,39.0,30.0,43.5,56.5
4,89060,30.0,15.0,15.0,50.0,50.0
5,89061,51.0,23.0,28.0,54.9,45.1
6,89439,80.0,41.0,39.0,48.8,51.2
...,...,...,...,...,...,...
1766,96148,788.0,425.0,363.0,46.1,53.9
1767,96150,NaN,NaN,NaN,47.0,53.0
1768,96155,8.0,5.0,3.0,37.5,62.5
1769,96161,NaN,NaN,NaN,47.6,52.4


In [9]:
sex_df = df.drop(columns=['Total population', 'Male', 'Female'])
sex_df

,Geography,Percent Population Female,Percent Population Male
2,89010,41.9,58.1
3,89019,43.5,56.5
4,89060,50.0,50.0
5,89061,54.9,45.1
6,89439,48.8,51.2
...,...,...,...
1766,96148,46.1,53.9
1767,96150,47.0,53.0
1768,96155,37.5,62.5
1769,96161,47.6,52.4


In [10]:
gender_df = sex_df.melt(id_vars=["Geography"], 
        var_name="sex_group", 
        value_name="qty_pct")
gender_df

,Geography,sex_group,qty_pct
0,89010,Percent Population Female,41.9
1,89019,Percent Population Female,43.5
2,89060,Percent Population Female,50.0
3,89061,Percent Population Female,54.9
4,89439,Percent Population Female,48.8
...,...,...,...
3533,96148,Percent Population Male,53.9
3534,96150,Percent Population Male,53.0
3535,96155,Percent Population Male,62.5
3536,96161,Percent Population Male,52.4


In [11]:
sexid_df = pd.DataFrame(['Percent Population Male', 'Percent Population Female'])
sexid_df.columns = ['Sex']
sexid_df

,Sex
0,Percent Population Male
1,Percent Population Female


In [12]:
sexid_df.reset_index(inplace=True)
sexid_df

,index,Sex
0,0,Percent Population Male
1,1,Percent Population Female


In [13]:
sexid_df.columns = ['sex_id', 'sex_group']
sexid_df

,sex_id,sex_group
0,0,Percent Population Male
1,1,Percent Population Female


In [14]:
gender_df['sex_group'] = gender_df['sex_group'].map(sexid_df.set_index('sex_group')['sex_id'])
#gender_df.columns = ['zip', 'sex_id', 'qty_pct']
gender_df

,Geography,sex_group,qty_pct
0,89010,1,41.9
1,89019,1,43.5
2,89060,1,50.0
3,89061,1,54.9
4,89439,1,48.8
...,...,...,...
3533,96148,0,53.9
3534,96150,0,53.0
3535,96155,0,62.5
3536,96161,0,52.4


In [15]:
gender_df.rename(columns={'Geography': 'zip', 'sex_group': 'sex_id'}, inplace=True)
gender_df

,zip,sex_id,qty_pct
0,89010,1,41.9
1,89019,1,43.5
2,89060,1,50.0
3,89061,1,54.9
4,89439,1,48.8
...,...,...,...
3533,96148,0,53.9
3534,96150,0,53.0
3535,96155,0,62.5
3536,96161,0,52.4


## Race Dataframes

In [16]:
census_file = "Data/2010Census_demographics.csv"

In [17]:
race_df = pd.read_csv(census_file, skiprows=8)
race_df.drop(0, inplace=True)
race_df.drop(1, inplace=True)

race_df_new = race_df.drop(['Unnamed: 2', 'Total Asian', 'Total NHOPI', 'Unnamed: 20'], axis=1)

race_df_new.rename(columns={"Unnamed: 1":"total_population", "Unnamed: 0":"zip","White": "white", "Black or African American": "black", "American Indian and Alaska Native":"native_american",
                            "Asian Indian":"indian", "Chinese":"chinese", "Filipino":"filipino", "Japanese":"japanese",
                            "Korean":"korean", "Vietnamese":"vietnamese", "Other Asian":"other_asian", "Native Hawaiian":"hawaiian",
                            "Guamanian or Chamorro":"guamanian", "Samoan":"samoan", "Other Pacific Islander":"pacific_islander",
                            "Some Other Race":"other"}, inplace=True)
race_df_new['zip'] = race_df_new['zip'].map(lambda x: x.lstrip('ZCTA5').rstrip('(California part only)'))

race_df_new['total_population'] = pd.to_numeric(race_df_new['total_population'].str.replace(',',''), errors='coerce')
race_df_new['white'] = pd.to_numeric(race_df_new['white'].str.replace(',',''), errors='coerce')
race_df_new['black'] = pd.to_numeric(race_df_new['black'].str.replace(',',''), errors='coerce')
race_df_new['native_american'] = pd.to_numeric(race_df_new['native_american'].str.replace(',',''), errors='coerce')
race_df_new['indian'] = pd.to_numeric(race_df_new['indian'].str.replace(',',''), errors='coerce')
race_df_new['chinese'] = pd.to_numeric(race_df_new['chinese'].str.replace(',',''), errors='coerce')
race_df_new['filipino'] = pd.to_numeric(race_df_new['filipino'].str.replace(',',''), errors='coerce')
race_df_new['japanese'] = pd.to_numeric(race_df_new['japanese'].str.replace(',',''), errors='coerce')
race_df_new['korean'] = pd.to_numeric(race_df_new['korean'].str.replace(',',''), errors='coerce')
race_df_new['vietnamese'] = pd.to_numeric(race_df_new['vietnamese'].str.replace(',',''), errors='coerce')
race_df_new['other_asian'] = pd.to_numeric(race_df_new['other_asian'].str.replace(',',''), errors='coerce')
race_df_new['hawaiian'] = pd.to_numeric(race_df_new['hawaiian'].str.replace(',',''), errors='coerce')
race_df_new['guamanian'] = pd.to_numeric(race_df_new['guamanian'].str.replace(',',''), errors='coerce')
race_df_new['samoan'] = pd.to_numeric(race_df_new['samoan'].str.replace(',',''), errors='coerce')
race_df_new['pacific_islander'] = pd.to_numeric(race_df_new['pacific_islander'].str.replace(',',''), errors='coerce')
race_df_new['other'] = pd.to_numeric(race_df_new['other'].str.replace(',',''), errors='coerce')


race_df_new['white'] = race_df_new['white']/race_df_new['total_population']
race_df_new['black'] = race_df_new['black']/race_df_new['total_population']
race_df_new['native_american'] = race_df_new['native_american']/race_df_new['total_population']
race_df_new['indian'] = race_df_new['indian']/race_df_new['total_population']
race_df_new['chinese'] = race_df_new['chinese']/race_df_new['total_population']
race_df_new['filipino'] = race_df_new['filipino']/race_df_new['total_population']
race_df_new['japanese'] = race_df_new['japanese']/race_df_new['total_population']
race_df_new['korean'] = race_df_new['korean']/race_df_new['total_population']
race_df_new['vietnamese'] = race_df_new['vietnamese']/race_df_new['total_population']
race_df_new['other_asian'] = race_df_new['other_asian']/race_df_new['total_population']
race_df_new['hawaiian'] = race_df_new['hawaiian']/race_df_new['total_population']
race_df_new['guamanian'] = race_df_new['guamanian']/race_df_new['total_population']
race_df_new['samoan'] = race_df_new['samoan']/race_df_new['total_population']
race_df_new['pacific_islander'] = race_df_new['pacific_islander']/race_df_new['total_population']
race_df_new['other'] = race_df_new['other']/race_df_new['total_population']

race_df_new = race_df_new.drop(['total_population'], axis=1)

race_df_melt = race_df_new.melt(id_vars=['zip'], var_name='race', value_name='percent_of_race')

race_df_melt['percent_of_race'] = race_df_melt['percent_of_race'].map("{:.1%}".format)

In [18]:
data = {'race_id': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], 
        'race_group': ["white", "black", "native_american",
                "indian", "chinese", "filipino", "japanese",
                "korean", "vietnamese", "other_asian", "hawaiian",
                "guamanian", "samoan", "pacific_islander","other"]}
race_table = pd.DataFrame(data=data)
race = race_table

race

,race_id,race_group
0,1,white
1,2,black
2,3,native_american
3,4,indian
4,5,chinese
5,6,filipino
6,7,japanese
7,8,korean
8,9,vietnamese
9,10,other_asian


In [19]:
race_df_melt = race_df_melt.replace({"race": {'white':1,'black':2,'native_american':3,'indian':4,
                               'chinese':5,'filipino':6,'japanese':7,'korean':8,
                               'vietnamese':9,'other_asian':10,'hawaiian':10,
                               'guamanian':12,'samoan':13,'pacific_islander':14,'other':15}}) 

In [20]:
race_df_melt = race_df_melt.rename(columns={"race": "race_id","percent_of_race":"qty_pct"})

In [47]:
race_df_melt.head()
race_zip = race_df_melt

race_zip

,zip,race_id,qty_pct
0,89010,1,58.1%
1,89019,1,79.7%
2,89060,1,90.0%
3,89061,1,82.4%
4,89439,1,97.5%
...,...,...,...
26530,96148,15,8.4%
26531,96150,15,11.4%
26532,96155,15,0.0%
26533,96161,15,8.3%


## Age Dateframes

In [22]:
# Read in sales data csv to Data Frame
census_df = pd.read_csv('Data/2010Census_age_gender.csv', skiprows=7)
census_df.drop(0, inplace=True)
census_df.drop(1, inplace=True)
census_df.head()

,Geography,Total population,Male,Female,Median age (years),Male Median age (years),Female Median age (years),Average Household size,Average Family size,Persons Under 5 years,Persons Under 18 years,Persons Age 21+,Persons Age 55+,Persons Age 60+,Persons Age 65+,Percent Population Female,Percent Population Less than 18,Unnamed: 17
2,ZCTA5 89010 (California part only),31,18,13,26.5,24.5,26.5,4.43,4.43,4,13,17,2,1,1,41.9%,41.9%,NaN
3,ZCTA5 89019 (California part only),69,39,30,38.3,38.5,38.0,2.88,3.93,8,18,49,17,11,6,43.5%,26.1%,NaN
4,ZCTA5 89060 (California part only),30,15,15,44.0,38.5,48.5,3.75,3.75,5,7,23,12,7,4,50.0%,23.3%,NaN
5,ZCTA5 89061 (California part only),51,23,28,47.5,40.5,50.0,2.43,2.92,2,13,38,19,16,13,54.9%,25.5%,NaN
6,ZCTA5 89439 (California part only),80,41,39,57.5,57.5,57.5,1.95,2.40,2,6,72,46,35,20,48.8%,7.5%,NaN


In [23]:
# Clean Zip code column
census_df['Geography'] = census_df['Geography'].map(lambda x: x.lstrip('ZCTA5').rstrip('(California part only)'))
# Drop unused columns
census_df_new = census_df.drop(['Median age (years)','Male Median age (years)', 'Female Median age (years)', 'Average Household size',
                                'Average Family size','Percent Population Female', 'Persons Under 5 years', 'Persons Under 18 years', 
                                'Unnamed: 17', 'Male', 'Female'], axis=1)
# Turn data into floats and calculate percentages
census_df_new['Persons Age 21+'] = pd.to_numeric(census_df_new['Persons Age 21+'].str.replace(',',''), errors='coerce')
census_df_new['Persons Age 55+'] = pd.to_numeric(census_df_new['Persons Age 55+'].str.replace(',',''), errors='coerce')
census_df_new['Persons Age 60+'] = pd.to_numeric(census_df_new['Persons Age 60+'].str.replace(',',''), errors='coerce')
census_df_new['Persons Age 65+'] = pd.to_numeric(census_df_new['Persons Age 65+'].str.replace(',',''), errors='coerce')
census_df_new['Total population'] = pd.to_numeric(census_df_new['Total population'].str.replace(',',''), errors='coerce')

census_df_new['21-55'] = census_df_new['Persons Age 21+'] / census_df_new['Total population']
census_df_new['55-60'] = census_df_new['Persons Age 55+'] / census_df_new['Total population']
census_df_new['60-65'] = census_df_new['Persons Age 60+'] / census_df_new['Total population']
census_df_new['65+'] = census_df_new['Persons Age 65+'] / census_df_new['Total population']

census_df_new.head()

,Geography,Total population,Persons Age 21+,Persons Age 55+,Persons Age 60+,Persons Age 65+,Percent Population Less than 18,21-55,55-60,60-65,65+
2,89010,31,17,2,1,1,41.9%,0.548387,0.064516,0.032258,0.032258
3,89019,69,49,17,11,6,26.1%,0.710145,0.246377,0.159420,0.086957
4,89060,30,23,12,7,4,23.3%,0.766667,0.400000,0.233333,0.133333
5,89061,51,38,19,16,13,25.5%,0.745098,0.372549,0.313725,0.254902
6,89439,80,72,46,35,20,7.5%,0.900000,0.575000,0.437500,0.250000


In [24]:
# Clean DataFrame to necessary data only
age_df = census_df_new.drop(['Total population','Persons Age 21+', 'Persons Age 55+', 'Persons Age 60+',
                                'Persons Age 65+'], axis=1)
age_df = age_df.rename(columns={"Percent Population Less than 18":"under_18"})

age_df['21-55'] = age_df['21-55'].map("{:.1%}".format)
age_df['55-60'] = age_df['55-60'].map("{:.1%}".format)
age_df['60-65'] = age_df['60-65'].map("{:.1%}".format)
age_df['65+'] = age_df['65+'].map("{:.1%}".format)

age_df.head()

,Geography,under_18,21-55,55-60,60-65,65+
2,89010,41.9%,54.8%,6.5%,3.2%,3.2%
3,89019,26.1%,71.0%,24.6%,15.9%,8.7%
4,89060,23.3%,76.7%,40.0%,23.3%,13.3%
5,89061,25.5%,74.5%,37.3%,31.4%,25.5%
6,89439,7.5%,90.0%,57.5%,43.8%,25.0%


In [25]:
# Melt data in DataFrame
age_df = age_df.melt(id_vars=['Geography'], var_name='age_group')
age_df.head()

,Geography,age_group,value
0,89010,under_18,41.9%
1,89019,under_18,26.1%
2,89060,under_18,23.3%
3,89061,under_18,25.5%
4,89439,under_18,7.5%


In [26]:
# Rename columns to match SQL Database age_zip table
age_df = age_df.rename(columns = {"Geography": "zip", 
                                 "value": "qty_pct"})
age_df

,zip,age_group,qty_pct
0,89010,under_18,41.9%
1,89019,under_18,26.1%
2,89060,under_18,23.3%
3,89061,under_18,25.5%
4,89439,under_18,7.5%
...,...,...,...
8840,96148,65+,10.5%
8841,96150,65+,10.0%
8842,96155,65+,25.0%
8843,96161,65+,8.1%


In [27]:
# Create age definitions DataFrame
age = age_df.drop(axis = 1, columns=['zip', 'qty_pct'])
age.drop_duplicates(subset = 'age_group', inplace = True)
age

,age_group
0,under_18
1769,21-55
3538,55-60
5307,60-65
7076,65+


In [28]:
# Final age DataFrame table
age.reset_index(inplace = True)
age

,index,age_group
0,0,under_18
1,1769,21-55
2,3538,55-60
3,5307,60-65
4,7076,65+


In [29]:
# Change column titles
age = age.rename(columns= {'index': 'age_id'})
age

,age_id,age_group
0,0,under_18
1,1769,21-55
2,3538,55-60
3,5307,60-65
4,7076,65+


In [30]:
# Final age_zip table with age_ids from age definitions table
age_df['age_group'] = age_df['age_group'].map(age.set_index('age_group')['age_id'])
age_df

,zip,age_group,qty_pct
0,89010,0,41.9%
1,89019,0,26.1%
2,89060,0,23.3%
3,89061,0,25.5%
4,89439,0,7.5%
...,...,...,...
8840,96148,7076,10.5%
8841,96150,7076,10.0%
8842,96155,7076,25.0%
8843,96161,7076,8.1%


## Product and Sales Dataframes

In [31]:
# Read in sales data csv to Data Frame
sales_df = pd.read_csv('Data/Sales_data.csv')
sales_df.head()

,Chain Name,Customer Name,Address,City,State,Zip Code,Prod #,Description,Pack,Size,Cost,Units
0,NaN,ANNS HEALTH FOOD,2634 S ZANG BLVD,DALLAS,TX,75224,14480,GNG LMN PRTN SMOOTHIE,12,12 OZ,$33.96,1
1,NaN,ATKINSONS MARKETS,91 E. CROY ST,HAILEY,ID,83333,45453,SUPERFOOD POWDER,1,20 OZ,$16.19,1
2,NaN,"BLOSSOM ON LOPEZ, INC.",295 VILLAGE RD,LOPEZ ISLAND,WA,98261,76740,NUT SPICES BAR,12,2 OZ,$15.12,1
3,NaN,BROULIMS SUPERMARKET,141 WASHINGTON ST,AFTON,WY,83110,98474,VANILLA PLANT YOGURT,6,6 OZ,$6.18,1
4,NaN,RJ`S MARKET,1425 SANSOME STREET,SAN FRANCISCO,CA,94111,76740,NUT SPICES BAR,12,2 OZ,$30.24,2


In [32]:
# Filter for CA only stores
ca_sales = sales_df.loc[sales_df['State'] == 'CA', :]
ca_sales.head()

,Chain Name,Customer Name,Address,City,State,Zip Code,Prod #,Description,Pack,Size,Cost,Units
4,NaN,RJ`S MARKET,1425 SANSOME STREET,SAN FRANCISCO,CA,94111,76740,NUT SPICES BAR,12,2 OZ,$30.24,2
5,NaN,RJ`S MARKET,1425 SANSOME STREET,SAN FRANCISCO,CA,94111,80760,CHOC PRTN BAR,12,2 OZ,$15.12,1
7,NaN,EUREKA AND HARVEST,2165 CENTRAL AVE,MCKINLEYVILLE,CA,95519,76740,NUT SPICES BAR,12,2 OZ,$15.12,1
8,NaN,GUS`S COMMUNITY MKT,2111 HARRISON ST,SAN FRANCISCO,CA,94110,14476,COCO BANA SMOOTHIE,12,12 OZ,$169.80,5
9,NaN,GUS`S COMMUNITY MKT,2111 HARRISON ST,SAN FRANCISCO,CA,94110,25452,BERRY PRTN SMOOTHIE,12,12 OZ,$33.96,1


In [33]:
# Clean sales data to create CA store list
ca_cleaned = ca_sales[['Customer Name', 'Address', 'City', 'State', 'Zip Code']]
ca_cleaned.sort_values('Customer Name', inplace=True)
ca_cleaned.drop_duplicates(subset = 'Customer Name', inplace = True)
ca_cleaned.reset_index(inplace = True)
ca_cleaned = ca_cleaned.rename(columns = {'Customer Name': 'store_name', 
                            'Address': 'address', 
                            'City': 'city', 
                            'State': 'state',
                            'Zip Code': 'zip', 
                            'index': 'store_id'})
ca_cleaned.head()

//anaconda3/envs/PythonData1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/PythonData1/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,store_id,store_name,address,city,state,zip
0,92,ANDYS CORTE MADERA,195 TAMAL VISTA BLVD,CORTE MADERA,CA,94925
1,93,ANDYS LOCAL MARKET,75 LOCH LOMOND DRIVE,SAN RAFAEL,CA,94901
2,160,ARCATA CO-OP,811 I STREET,ARCATA,CA,95521
3,46,ARTESIAN HEALTH FOODS II,2251 W. GRANT LINE RD,TRACY,CA,95377
4,208,BEL AIR #502,4320 ARDEN WAY,SACRAMENTO,CA,95864


In [34]:
# Clean sales data to create product table
products_df = sales_df[['Prod #', 'Description', 'Pack', 'Size', 'Cost', 'Units']]
products_df.sort_values('Prod #', inplace=True)
products_df.drop_duplicates(subset = 'Prod #', inplace=True)
products_df.reset_index(inplace = True)
products_df = products_df.rename(columns = {'Prod #': 'product_number', 
                                           'Description': 'description', 
                                           'Pack': 'pack_qty', 
                                           'Size': 'size', 
                                           'index': 'product_id'})
products_df

//anaconda3/envs/PythonData1/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
//anaconda3/envs/PythonData1/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,product_id,product_number,description,pack_qty,size,Cost,Units
0,11,14476,COCO BANA SMOOTHIE,12,12 OZ,$33.96,1
1,57,14480,GNG LMN PRTN SMOOTHIE,12,12 OZ,$33.96,1
2,65,25452,BERRY PRTN SMOOTHIE,12,12 OZ,$203.76,6
3,272,35454,PLAIN PLANT YOGURT,6,6 OZ,$6.18,1
4,224,45453,SUPERFOOD POWDER,1,20 OZ,$16.19,1
5,28,56732,GREEN PRTN SMOOTHIE,12,12 OZ,$33.96,1
6,171,60678,CITRUS SMOOTHIE,12,12 OZ,$33.96,1
7,30,66736,HONEY BUTTER SMOOTHIE,12,12 OZ,$33.96,1
8,13,70679,FRUIT BERRY BAR,12,2 OZ,$15.12,1
9,55,76740,NUT SPICES BAR,12,2 OZ,$15.12,1


In [35]:
# Find price of 1 unit of each product
products_df['Cost'] = products_df['Cost'].str[1:]
products_df['Cost'] = pd.to_numeric(products_df['Cost'], errors='coerce')
products_df['price'] = products_df['Cost'] / products_df['Units']
products_df

,product_id,product_number,description,pack_qty,size,Cost,Units,price
0,11,14476,COCO BANA SMOOTHIE,12,12 OZ,33.96,1,33.96
1,57,14480,GNG LMN PRTN SMOOTHIE,12,12 OZ,33.96,1,33.96
2,65,25452,BERRY PRTN SMOOTHIE,12,12 OZ,203.76,6,33.96
3,272,35454,PLAIN PLANT YOGURT,6,6 OZ,6.18,1,6.18
4,224,45453,SUPERFOOD POWDER,1,20 OZ,16.19,1,16.19
5,28,56732,GREEN PRTN SMOOTHIE,12,12 OZ,33.96,1,33.96
6,171,60678,CITRUS SMOOTHIE,12,12 OZ,33.96,1,33.96
7,30,66736,HONEY BUTTER SMOOTHIE,12,12 OZ,33.96,1,33.96
8,13,70679,FRUIT BERRY BAR,12,2 OZ,15.12,1,15.12
9,55,76740,NUT SPICES BAR,12,2 OZ,15.12,1,15.12


In [36]:
# Final product table 
products_df = products_df.drop(axis = 1, columns=['Cost', 'Units'])
products_df

,product_id,product_number,description,pack_qty,size,price
0,11,14476,COCO BANA SMOOTHIE,12,12 OZ,33.96
1,57,14480,GNG LMN PRTN SMOOTHIE,12,12 OZ,33.96
2,65,25452,BERRY PRTN SMOOTHIE,12,12 OZ,33.96
3,272,35454,PLAIN PLANT YOGURT,6,6 OZ,6.18
4,224,45453,SUPERFOOD POWDER,1,20 OZ,16.19
5,28,56732,GREEN PRTN SMOOTHIE,12,12 OZ,33.96
6,171,60678,CITRUS SMOOTHIE,12,12 OZ,33.96
7,30,66736,HONEY BUTTER SMOOTHIE,12,12 OZ,33.96
8,13,70679,FRUIT BERRY BAR,12,2 OZ,15.12
9,55,76740,NUT SPICES BAR,12,2 OZ,15.12


In [37]:
# Create store_sales table
sales_table = ca_sales[['Customer Name', 'Prod #', 'Units', 'Zip Code']]

sales_table = sales_table.sort_values('Customer Name')
sales_table.head()

,Customer Name,Prod #,Units,Zip Code
92,ANDYS CORTE MADERA,25452,1,94925
93,ANDYS LOCAL MARKET,25452,1,94901
160,ARCATA CO-OP,45453,7,95521
46,ARTESIAN HEALTH FOODS II,45453,1,95377
208,BEL AIR #502,45453,1,95864


In [38]:
# Replace customer name with store_id from store table
sales_table['Customer Name'] = sales_table['Customer Name'].map(ca_cleaned.set_index('store_name')['store_id'])
sales_table

,Customer Name,Prod #,Units,Zip Code
92,92,25452,1,94925
93,93,25452,1,94901
160,160,45453,7,95521
46,46,45453,1,95377
208,208,45453,1,95864
...,...,...,...,...
186,187,76740,2,95482
53,53,35454,3,93023
54,53,45453,3,93023
418,418,76740,1,93534


In [39]:
# Replace product # with product_id from product table
sales_table['Prod #'] = sales_table['Prod #'].map(products_df.set_index('product_number')['product_id'])
sales_table.head()

,Customer Name,Prod #,Units,Zip Code
92,92,65,1,94925
93,93,65,1,94901
160,160,224,7,95521
46,46,224,1,95377
208,208,224,1,95864


In [40]:
# Rename column titles to have final store_sales table
sales_table = sales_table.rename(columns = {'Customer Name': 'store_id', 
                                 'Zip Code': 'zip', 
                                 'Prod #': 'product_id', 
                                'Units': 'units'})
sales_table

,store_id,product_id,units,zip
92,92,65,1,94925
93,93,65,1,94901
160,160,224,7,95521
46,46,224,1,95377
208,208,224,1,95864
...,...,...,...,...
186,187,55,2,95482
53,53,272,3,93023
54,53,224,3,93023
418,418,55,1,93534


## Create Database Connection

In [41]:
#connection_string = "postgres:postgres@localhost:5432/customer_db"
connection_string = 'postgres:d@Tafnd.1357@localhost:5432/ETL-Project'
engine = create_engine(f'postgresql://{connection_string}')

## Load DataFrames into Database

In [64]:
# sex Table
sexid_df.to_sql(name='sex', con=engine, if_exists='append', index=False)

In [49]:
# race Table
race.to_sql(name='race', con=engine, if_exists='append', index=False)

In [54]:
# age table
age.to_sql(name='age', con=engine, if_exists='append', index=False)

In [55]:
# product table
products_df.to_sql(name='product', con=engine, if_exists='append', index=False)

In [56]:
# store table
ca_cleaned.to_sql(name='store', con=engine, if_exists='append', index=False)

In [58]:
# sex_zip Table
gender_df.to_sql(name='sex_zip', con=engine, if_exists='append', index=False)

In [59]:
# race_zip Table
race_zip.to_sql(name='race_zip', con=engine, if_exists='append', index=False)

In [62]:
# age_zip table
age_df.to_sql(name='age_zip', con=engine, if_exists='append', index=False)

In [63]:
# store sales table
sales_table.to_sql(name='store_sales', con=engine, if_exists='append', index=False)